In [1]:
%matplotlib inline
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 16
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm

In [7]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
NUM_CATEGORIES = len(CATEGORIES)

SEED = 1987
data_dir = 'seeds'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

In [3]:
# import glob
# from PIL import Image

# # train_paths = glob.glob(train_dir + '/*/*')
# # print(len(train_paths))

# test_paths = glob.glob(test_dir + '/*')
# print(len(test_paths))

# sizes = []
# for x in test_paths:
#     sizes.append(Image.open(x).convert('RGB').size)
    
# # ratios = [x[0]*1.0/x[1] for x in sizes]
# sizes.sort(key=lambda x: x[0] * x[1])
# for x in sizes[:5]:
#     print(x)
# for x in sizes[-5:]:
#     print(x)

# COUNT_SIZES = [0, 64, 96, 128, 160, 192, 224, 256, 512, 1024, 2048, 4096]
# buckets = {x: [] for x in COUNT_SIZES}

# for size in sizes:
#     w,h = size
#     for i in range(len(COUNT_SIZES) - 1):
#         if COUNT_SIZES[i] <= max(w,h) < COUNT_SIZES[i+1]:
#             buckets[COUNT_SIZES[i]].append(size)

# x = COUNT_SIZES
# y = [len(buckets[x]) for x in buckets]

# ind = np.arange(len(COUNT_SIZES))  # the x locations for the groups
# fig, ax = plt.subplots(figsize=(20,6))
# rect = ax.bar(ind, y)
# ax.set_xticks(ind)
# ax.set_xticklabels(COUNT_SIZES)
    
# plt.show()

# s = 0
# n = len(sizes)
# for i in range(len(COUNT_SIZES) - 1):
#     print("[{}, {})\t{:.2f}%".format(COUNT_SIZES[i], COUNT_SIZES[i+1], (s+len(buckets[COUNT_SIZES[i]]))*100.0/n))

In [8]:
import glob
import random

def generate_training_txt():
    random.seed(0)

    train_data = {}
    val_data = {}
    n_val = 50
    
    for cat in CATEGORIES:
        x = glob.glob(os.path.join(train_dir, cat) + '/*')
        random.shuffle(x)
        
        train_data[cat] = x[n_val:]
        val_data[cat] = x[:n_val]

    for data, fname in [(train_data, 'train'),(val_data, 'test')]:
        with open(os.path.join(data_dir, '{}.txt'.format(fname)), 'w') as f:
            for i,cat in enumerate(CATEGORIES):
                for x in data[cat]:
                    f.write("{}\t{}\n".format(x, cat))
                    
# generate_training_txt()

In [9]:
import sys
sys.path.insert(0, '/home/Yi/codes/fast_train_classification/')

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [10]:
from train_clf_net import init_net, PathDataset

pretrained_weights_path = '/home/Yi/seeds/weights/15_0.88.pth'
net = init_net(pretrained_weights_path, n_classes=len(CATEGORIES), net_arch='resnet50',if_strict=True, if_parallel=False)


net.eval()

ModuleNotFoundError: No module named 'train_clf_net'

In [11]:
from torchvision import transforms

input_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
    
def resize_wo_upsample(pil):
    w,h = pil.size
    if w > 224 or h > 224:
        return transforms.Resize((input_size, input_size))(pil)
    
    return pil
    
test_paths = glob.glob(test_dir + '/*')
dataset = PathDataset(test_paths, transforms.Compose([
#         transforms.Resize((input_size, input_size)),
        resize_wo_upsample,
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]))

print(len(dataset))

NameError: name 'PathDataset' is not defined

In [3]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

pred = None
loader = DataLoader(dataset, batch_size=1, shuffle=False,
        num_workers=2, pin_memory=True)

net.eval()
for i, inputs in enumerate(loader):
    with torch.no_grad():
        inputs = inputs.cuda()  # inputs.to(device)
#         out = net(inputs)
        
#         features = net.features(inputs)
#         out = F.relu(features, inplace=True)
#         n,c,h,w = out.size()
#         out = F.avg_pool2d(out, kernel_size=(h,w), stride=1).view(features.size(0), -1)
#         out = net.classifier(out)
    
        x = net.conv1(inputs)
        x = net.bn1(x)
        x = net.relu(x)
        x = net.maxpool(x)

        x = net.layer1(x)
        x = net.layer2(x)
        x = net.layer3(x)
        x = net.layer4(x)
        n,c,h,w = x.size()
        x = F.avg_pool2d(x, kernel_size=(h,w), stride=1).view(x.size(0), -1)
        out = net.fc(x)
        
        if pred is None:
            pred = out.data
        else:
            pred = torch.cat((pred, out), 0)
            
print(pred)

NameError: name 'dataset' is not defined

In [13]:
print(pred.size())

import torch.nn.functional as F
probs, cls = pred.max(dim=1)
print(probs.size(), cls.size())

torch.Size([794, 12])
torch.Size([794]) torch.Size([794])


In [15]:
labels = {}

for line in open('/home/Yi/seeds/weights/labels.txt').readlines():
    segs = line.strip().split(' ')
    i = int(segs[0])
    cat = ' '.join(segs[1:])
    labels[i] = cat
    
print(CATEGORIES)
print(labels)

submit_id = '11061835'
output_path = "/home/Yi/seeds/submits/{}.csv".format(submit_id)
with open(output_path, 'w') as f:
    f.write("file,species\n")
    for x,i in zip(test_paths, cls.cpu().numpy()):
        f.write("{},{}\n".format(os.path.basename(x), labels[i]))

['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
{0: 'Black-grass', 1: 'Charlock', 2: 'Cleavers', 3: 'Common Chickweed', 4: 'Common wheat', 5: 'Fat Hen', 6: 'Loose Silky-bent', 7: 'Maize', 8: 'Scentless Mayweed', 9: 'Shepherds Purse', 10: 'Small-flowered Cranesbill', 11: 'Sugar beet'}


In [16]:
from collections import Counter
c = Counter(cls.cpu().numpy())
print(c)

Counter({6: 121, 3: 110, 8: 91, 11: 85, 10: 84, 5: 64, 1: 61, 2: 39, 9: 38, 7: 37, 0: 32, 4: 32})


In [ ]:
import shutil

dir_path = lambda x: "/home/Yi/seeds/outs/{}".format(x.replace(' ', '_'))
for x in CATEGORIES:
    os.system("rm -r {}".format(dir_path(x)))
    os.mkdir(dir_path(x))

for x,i in zip(test_paths, cls.cpu().numpy()):
    c = labels[i]
    shutil.copy(x, "{}/{}".format(dir_path(c), os.path.basename(x)))

In [ ]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 40
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp


import cv2
image = cv2.imread(train_paths[9])


image_mask = create_mask_for_plant(image)
image_segmented = segment_plant(image)
image_sharpen = sharpen_image(image_segmented)


fig, axs = plt.subplots(1, 4, figsize=(20, 20))
axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[1].imshow(image_mask)
axs[2].imshow(image_segmented)
axs[3].imshow(image_sharpen)

In [ ]:
lowThreshold = 0
max_lowThreshold = 100
ratio = 3
kernel_size = 3

def CannyThreshold(img, lowThreshold):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    detected_edges = cv2.GaussianBlur(gray,(3,3),0)
    detected_edges = cv2.Canny(detected_edges, lowThreshold, lowThreshold*ratio, apertureSize = kernel_size)
    dst = cv2.bitwise_and(img,img,mask = detected_edges)  # just add some colours to edges from original image.
    return dst

dst = CannyThreshold(image, 0)

# fig, axs = plt.subplots(1, 4, figsize=(20, 20))
# axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# axs[1].imshow(image_mask)
# axs[2].imshow(image_segmented)
# axs[3].imshow(image_sharpen)

plt.imshow(dst)
plt.show()